In [1]:
import sys
sys.path.append('../build')   # assume an out-of-source build

import numpy as np
import exafmm as fmm
import exafmm.modified_helmholtz as mod_helm

In [2]:
mod_helm.__doc__

"exafmm's submodule for Modified Helmholtz kernel"

### Create sources and targets

In [3]:
nsrcs = 100000
ntrgs = 100000

# generate random positions for particles
src_coords = np.random.random((nsrcs, 3))
trg_coords = np.random.random((nsrcs, 3))

# generate random charges for sources
src_charges = np.random.random(nsrcs)

In [4]:
# create a list of source instances
sources = mod_helm.init_sources(src_coords, src_charges)

# create a list of target instances
targets = mod_helm.init_targets(trg_coords)

### Create a ModifiedHelmholtzFMM instance

In [5]:
p = 10
ncrit = 400
depth = 4
wavek = 10
fmm = mod_helm.ModifiedHelmholtzFMM(p, ncrit, depth, wavek)

### Setup FMM

Given sources, targets and FMM configurations, `setup()` builds the tree and interaction list and pre-compute invariant matrices.

Set `skip_P2P` to `True` to skip near-field interactions for BEM applications.

In [6]:
skip_P2P = False
tree = mod_helm.setup(sources, targets, fmm, skip_P2P)

### Evaluate

`evaluate()` triggers the evaluation and returns a $n_{trg} \times 4$ `numpy.ndarray`.
The $i$-th row of `trg_values` starts with the potential value of the $i$-th target, followed by its three gradient values.

In [7]:
trg_values = mod_helm.evaluate(tree, fmm)

P2M                  : 4.1670400e-01
M2M                  : 1.2065800e-01
P2L                  : 7.0000000e-05
M2P                  : 5.7000000e-05
P2P                  : 4.1933200e-01
M2L                  : 9.0916700e-01
L2L                  : 1.1345400e-01
L2P                  : 4.5863600e-01


In [8]:
trg_values[6]

array([ 2824.1209738 ,  -634.31429962,   877.44886585, 19433.44884345])

### Check Accuracy

`fmm.verify(tree.leafs)` returns L2-norm the relative error of potential and gradient in a list. It only works when `skip_P2P` is set to `False`.

In [9]:
fmm.verify(tree.leafs)

[5.803627296485308e-11, 8.405424264161787e-09]

### Update charges of sources and run FMM iteratively

In [10]:
niters = 10

for i in range(niters):
    print('-'*10 + ' iteration {} '.format(i) + '-'*10)  # print divider between iterations
    
    src_charges = np.random.random(nsrcs)         # generate new random charges          
    mod_helm.update_charges(tree, src_charges)      # update charges
    mod_helm.clear_values(tree)                     # clear values
    trg_values = mod_helm.evaluate(tree, fmm)       # evaluate potentials

    print("Error: ", fmm.verify(tree.leafs))       # check accuracy

---------- iteration 0 ----------
P2M                  : 4.1045300e-01
M2M                  : 1.4112000e-01
P2L                  : 1.6500000e-04
M2P                  : 3.9000000e-05
P2P                  : 4.1704300e-01
M2L                  : 9.1585100e-01
L2L                  : 1.4150900e-01
L2P                  : 4.5614800e-01
Error:  [7.343935290089744e-11, 7.69520274751419e-09]
---------- iteration 1 ----------
P2M                  : 3.5447600e-01
M2M                  : 1.2598900e-01
P2L                  : 6.9000000e-05
M2P                  : 6.0000000e-05
P2P                  : 4.1825500e-01
M2L                  : 9.2252700e-01
L2L                  : 1.3097200e-01
L2P                  : 4.5442500e-01
Error:  [4.0006900716248965e-11, 6.430789206135463e-09]
---------- iteration 2 ----------
P2M                  : 3.5858500e-01
M2M                  : 1.2269400e-01
P2L                  : 7.1000000e-05
M2P                  : 5.6000000e-05
P2P                  : 4.1898300e-01
M2L        